In [66]:
import torch
import torchvision.models.video as models
import torch.optim as optim
from tqdm import tqdm
from FitnessData import FitnessData
from torch.utils.data import DataLoader, random_split

In [59]:
model = models.r2plus1d_18(pretrained=True)

In [60]:
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=22)

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()  # Standard loss for classification
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [62]:
def train(model, dataloader, criterion, optimizer, num_epoch=10):
    model.train()
    for i in range(num_epoch):
        running_loss = 0.0
        correct = 0
        total = 0
        
        loop = tqdm(dataloader, leave=True)
        for video, label in loop:
            video, label = video.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(video)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += (predicted == label).sum().item()
            total += label.size(0)

            loop.set_description(f"Epoch [{i+1}/{num_epoch}]")
            loop.set_postfix(loss=running_loss / total, acc=100. * correct / total)

    print(f"Finished Training. Final Accuracy: {100 * correct / total:.2f}%")

In [63]:
dataset = FitnessData(root_dir="workoutfitness-video", transform=None, frames_per_clip=16)
train_size= int(0.7*(len(dataset)))
val_size = int(0.15*(len(dataset)))
test_size= len(dataset) - train_size - val_size

In [64]:
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [65]:
print(train_data[0][0].shape)

torch.Size([16, 3, 224, 224])


In [22]:
print(f"Train_size: {train_size}, Val_size: {val_size}, Test_size: {test_size}")

Train_size: 414, Val_size: 88, Test_size: 90


In [24]:
# Define batch size
batch_size = 4

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)


In [25]:
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(device), labels.to(device)
            outputs = model(videos)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total  # Return accuracy percentage

In [ ]:
train(model=model, dataloader=train_loader, criterion=criterion, optimizer=optimizer, num_epoch=10)